# Label Classifier (dSprites): Data Collection

This notebook contains code used to collect experiments data for various shift detection methods discussed in Lipton's paper. We compare their performance with the novel method that we created for detecting shift using the concept activation.

In [ ]:
## Solve dependency problem
# !git clone https://github.com/josipd/torch-two-sample.git
# %cd torch-two-sample
# !python setup.py install

In [2]:
import numpy as np
import torch
import random
# from torch_two_sample import *
from scipy.stats import ks_2samp, binom_test, chisquare, chi2_contingency, anderson_ksamp
from scipy.spatial import distance
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import scipy.io
from math import ceil
from copy import deepcopy
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import optimizers
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from tensorflow.python import keras
from tensorflow.python.keras import layers
from tqdm.notebook import tqdm
import pickle
from IPython.display import display, Markdown, Latex
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.random_projection import SparseRandomProjection

# Remove warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Mount drive (if needed)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
# Run utility functions
%run drive/MyDrive/Colab\ Notebooks/MPhil\ Project/Utilities.ipynb
%run drive/MyDrive/Colab\ Notebooks/MPhil\ Project/Experiment\ Utilities.ipynb

## Load Dataset

In [5]:
SEED = 20
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

# Load dataset, split into train, validation, and test sets
path = 'drive/MyDrive/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz'
x_train, x_test, y_train, y_test, c_train, c_test = load_dsprites(path, 100000, train_size=0.85, class_index=1)

# Split training into validation set as well 
x_train, x_valid = x_train[:70000], x_train[70000:]
y_train, y_valid = y_train[:70000], y_train[70000:]
c_train, c_valid = c_train[:70000], c_train[70000:]

Training samples: 85000
Testing samples: 15000


In [7]:
# Reshape to appropriate shift input
# It is noteworthy that for efficiency, we represent the images as only 2 dimension
# when we preprocessing (number of instances/ batch size * flatten size).
# When visualising back the image, we need to reshape it back to the original dimension
ORIGINAL_SHAPE = x_test.shape[1:] # constant hold the image original shape
x_test_flatten = deepcopy(x_test.reshape(x_test.shape[0], -1))
x_train_flatten = deepcopy(x_train.reshape(x_train.shape[0], -1))
x_valid_flatten = deepcopy(x_valid.reshape(x_valid.shape[0], -1))

## Dimensionality Reduction

This section built various dimensionality reduction methods, shown as follows:
- Concept bottleneck model (CBM)
- Trained and Untrained autoencoders (TAE and UAE)
- Principal component analysis (PCA)
- Sparse random projection (SRP)

### Concept Bottleneck Models



#### Input to Concept Model

In this section, we will employ the multitask model. Nevertheless, we could use other models as listed below as a substitute:
1. **Multitask Model**: model with shared convolutional layers and multiple heads for respective task to predict color, shape, scale, rotation, x and y positions.
2. **Ensemble Model**: concatenation of six individual models, where each predicts color, shape, scale, rotation, x and y positions respectively.
3. **Binary Model**: model where concept values are binaries (e.g., last layers comprised neurons with sigmoid activation functions.

In [ ]:
multitask_model = MultitaskModel()
optimizer = tf.keras.optimizers.Adam(lr=1e-4, amsgrad=True)
multitask_model.compile(optimizer=optimizer,
                    loss=[
                        tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                        tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                        tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                        tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                        tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                        tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
                    ], metrics=["accuracy"])
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)

history = multitask_model.fit(x=x_train, y=[c_train[:, 0], c_train[:, 1], c_train[:, 2],
                        c_train[:, 3], c_train[:, 4], c_train[:, 5]], 
                        epochs=1000, batch_size=128,
                        validation_data=(x_valid, [c_valid[:, 0], c_valid[:, 1], c_valid[:, 2],
                            c_valid[:, 3], c_valid[:, 4], c_valid[:, 5]]),
                        callbacks=[lr_reducer, early_stopper])

Epoch 1/1000
547/547 [==============================] - 18s 23ms/step - loss: 13.7120 - color_loss: 0.0000e+00 - shape_loss: 1.1477 - scale_loss: 1.7765 - rotation_loss: 3.7951 - x_loss: 3.4979 - y_loss: 3.4948 - color_accuracy: 0.0000e+00 - shape_accuracy: 0.3670 - scale_accuracy: 0.2444 - rotation_accuracy: 0.0248 - x_accuracy: 0.0463 - y_accuracy: 0.0489 - val_loss: 12.8184 - val_color_loss: 0.0000e+00 - val_shape_loss: 1.0380 - val_scale_loss: 1.5115 - val_rotation_loss: 3.7031 - val_x_loss: 3.2845 - val_y_loss: 3.2813 - val_color_accuracy: 0.0000e+00 - val_shape_accuracy: 0.4391 - val_scale_accuracy: 0.4078 - val_rotation_accuracy: 0.0257 - val_x_accuracy: 0.1431 - val_y_accuracy: 0.1369
Epoch 2/1000
547/547 [==============================] - 12s 22ms/step - loss: 12.7149 - color_loss: 0.0000e+00 - shape_loss: 1.0813 - scale_loss: 1.4813 - rotation_loss: 3.8247 - x_loss: 3.1645 - y_loss: 3.1631 - color_accuracy: 0.0000e+00 - shape_accuracy: 0.4218 - scale_accuracy: 0.3497 - rotati

In [ ]:
## Run this cell to save models
# Save whole model
path = "drive/MyDrive/multitask_saved_model/"
multitask_model.save(path)

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(231)
ax2 = fig.add_subplot(232)
ax3 = fig.add_subplot(233)
ax4 = fig.add_subplot(234)
ax5 = fig.add_subplot(235)
ax6 = fig.add_subplot(236)
axes = [ax1, ax2, ax3, ax4, ax5, ax6]

# Plot training validation loss
ax1.plot(history.history["color_loss"], label="Training Loss")
ax1.plot(history.history["val_color_loss"], label="Validation Loss")

ax2.plot(history.history["shape_loss"], label="Training Loss")
ax2.plot(history.history["val_shape_loss"], label="Validation Loss")

ax3.plot(history.history["scale_loss"], label="Training Loss")
ax3.plot(history.history["val_scale_loss"], label="Validation Loss")

ax4.plot(history.history["rotation_loss"], label="Training Loss")
ax4.plot(history.history["val_rotation_loss"], label="Validation Loss")

ax5.plot(history.history["x_loss"], label="Training Loss")
ax5.plot(history.history["val_x_loss"], label="Validation Loss")

ax6.plot(history.history["y_loss"], label="Training Loss")
ax6.plot(history.history["val_y_loss"], label="Validation Loss")

for ax in axes:
    ax.legend()
    ax.grid(True)
plt.show()

In [ ]:
# Evaluation
concept_names = ["color", "shape", "scale", "rotation", "x", "y"]
for i, pred in enumerate(multitask_model.predict(x_test)):
    print("*"*20, f"Model: {concept_names[i]}", "*"*20)
    c_truth = c_test[:, i]
    c_pred = np.argmax(pred, axis=1)
    
    print(classification_report(c_truth, c_pred))
    print(confusion_matrix(c_truth, c_pred))
    print("\n\n")

******************** Model: color ********************
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     15000

    accuracy                           1.00     15000
   macro avg       1.00      1.00      1.00     15000
weighted avg       1.00      1.00      1.00     15000

[[15000]]



******************** Model: shape ********************
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5051
           1       1.00      1.00      1.00      4965
           2       1.00      1.00      1.00      4984

    accuracy                           1.00     15000
   macro avg       1.00      1.00      1.00     15000
weighted avg       1.00      1.00      1.00     15000

[[5051    0    0]
 [   0 4965    0]
 [   2    0 4982]]



******************** Model: scale ********************
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2554
 

#### Concept to Output Model

In this section, we will train concept to output model. The model will take concepts as input and shape as output. We follow *Independent Bottleneck* training procedure.

In [ ]:
# Build and train model
com = LogisticRegression()
com.fit(c_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_test_pred = com.predict(c_test)
print(classification_report(y_test_pred, y_test))
print(confusion_matrix(y_test_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5091
           1       1.00      1.00      1.00      5017
           2       1.00      1.00      1.00      4892

    accuracy                           1.00     15000
   macro avg       1.00      1.00      1.00     15000
weighted avg       1.00      1.00      1.00     15000

[[5091    0    0]
 [   0 5017    0]
 [   0    0 4892]]


### Principal Component Analysis

In [8]:
pca, n_components = principal_components_analysis(x_train_flatten)
print(f"The number of components to explain 80% of variance is {n_components}.")

The number of components to explain 80% of variance is 42.


### Sparse Random Projection

In [9]:
srp, n_components = sparse_random_projection(x_train_flatten)
print(f"The number of components to explain 80% of variance is {n_components}.")

The number of components to explain 80% of variance is 42.


In [13]:
x_valid_flatten.shape

(15000, 4096)

## Data Collection

This section performs various experiments to collect data using various methods further discussed in paper and thesis.

In [10]:
# Load model
path = "drive/MyDrive/multitask_saved_model/"
multitask_model = keras.models.load_model(path)

In [22]:
## Sanity check (ensure no false positive)
# For original data, no shift should be detected
test_statistic, p_val, detection_result = single_experiment(multitask_model,
                                                            "CBSDh",
                                                            x_valid, y_valid,
                                                            x_test_flatten[:300],
                                                            y_test[:300],
                                                            ORIGINAL_SHAPE)
print(detection_result)

{'color': 0, 'shape': 0, 'scale': 0, 'rotation': 0, 'x': 0, 'y': 0}


### CBSDs

Use the softmax outputs of each concept predictions as the reduced representation.

In [23]:
method = "CBSDs"

#### Knockout Shift

In [25]:
shift_type = "ko"
shift_type_params = {"cl": "majority"}

In [28]:
dict_result = dict()
dict_result["test"] = do_experiment_per_shift(multitask_model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
dict_result["valid"] = do_experiment_per_shift(multitask_model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)

In [29]:
save_dict_result(shift_type, method, dict_result)

Saving successfully.


#### Gaussian Shift

In [ ]:
shift_type = "gaussian"
shift_type_params = None

In [ ]:
dict_result = dict()
dict_result["test"] = do_experiment_per_shift(multitask_model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
dict_result["valid"] = do_experiment_per_shift(multitask_model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)

In [ ]:
save_dict_result(shift_type, method, dict_result)

Saving successfully.


#### Concept Shift

We consider the following concept shift combinations:
- Scale
- Scale + Shape
- Position X
- Position Y
- Position X + Position Y
- Position X + Position Y + Scale

In [ ]:
# List of shift_type and params
shift_type_list = [
                   "concept_scale",
                   ["concept_shape", "concept_scale"],
                   "concept_x",
                   "concept_y",
                   ["concept_x", "concept_y"],
                   ["concept_x", "concept_y", "scale"],
]

shift_type_params_list = [
                          {"cl": "majority", "concept_idx": 2},
                          [{"cl": "majority", "concept_idx": 1}, {"cl": "majority", "concept_idx": 2}],
                          {"cl": "majority", "concept_idx": 4},
                          {"cl": "majority", "concept_idx": 5},
                          [{"cl": "majority", "concept_idx": 4}, {"cl": "majority", "concept_idx": 5}],
                          [{"cl": "majority", "concept_idx": 2}, {"cl": "majority", "concept_idx": 4}, {"cl": "majority", "concept_idx": 5}],
]

In [ ]:
# Iterate over possible shift type, conduct experimentation, store results
for shift_type, shift_type_params in zip(shift_type_list, shift_type_params_list):
    dict_result = dict()

    # Conduct experimentation
    dict_result["test"] = do_experiment_per_shift(multitask_model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    dict_result["valid"] = do_experiment_per_shift(multitask_model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    
    # Save result
    save_dict_result(shift_type, method, dict_result)


Saving successfully.


#### Image Shift

We consider the following concept shift combinations:
- rotation
- shear
- zoom
- translation (x)
- translation (y)
- translation (x, y)
- img (combination of all shifts)

In [ ]:
# List of shift_type and params
shift_type_list = [
                #    "rotation",
                #    "shear",
                #    "zoom",
                #    "width_shift",
                #    "height_shift",
                   ["width_shift", "height_shift"],
                   "img"
]

shift_type_params_list = [
                        #   {"orig_dims": ORIGINAL_SHAPE},
                        #   {"orig_dims": ORIGINAL_SHAPE},
                        #   {"orig_dims": ORIGINAL_SHAPE},
                        #   {"orig_dims": ORIGINAL_SHAPE},
                        #   {"orig_dims": ORIGINAL_SHAPE},
                          [{"orig_dims": ORIGINAL_SHAPE}, {"orig_dims": ORIGINAL_SHAPE}],
                          {"orig_dims": ORIGINAL_SHAPE},
]

In [ ]:
# Iterate over possible shift type, conduct experimentation, store results
for shift_type, shift_type_params in zip(shift_type_list, shift_type_params_list):
    dict_result = dict()

    # Conduct experimentation
    dict_result["test"] = do_experiment_per_shift(multitask_model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    dict_result["valid"] = do_experiment_per_shift(multitask_model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    
    # Save result
    save_dict_result(shift_type, method, dict_result)


Saving successfully.



Saving successfully.


### CBSDh
Use the hard outputs of each concept predictions as the reduced representation.

In [ ]:
method = "CBSDh"

#### Knockout Shift

In [ ]:
shift_type = "ko"
shift_type_params = {"cl": 0}

In [ ]:
dict_result = dict()
dict_result["test"] = do_experiment_per_shift(multitask_model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
dict_result["valid"] = do_experiment_per_shift(multitask_model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)

In [ ]:
save_dict_result(shift_type, method, dict_result)

Saving successfully.


#### Gaussian Shift

In [ ]:
shift_type = "gaussian"
shift_type_params = None

In [ ]:
dict_result = dict()
dict_result["test"] = do_experiment_per_shift(multitask_model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
dict_result["valid"] = do_experiment_per_shift(multitask_model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)

In [ ]:
save_dict_result(shift_type, method, dict_result)

Saving successfully.


#### Concept Shift

We consider the following concept shift combinations:
- Scale
- Scale + Shape
- Position X
- Position Y
- Position X + Position Y
- Position X + Position Y + Scale

In [ ]:
# List of shift_type and params
shift_type_list = [
                #    "concept_scale",
                #    ["concept_shape", "concept_scale"],
                #    "concept_x",
                #    "concept_y",
                #    ["concept_x", "concept_y"],
                   ["concept_x", "concept_y", "scale"],
]

shift_type_params_list = [
                        #   {"cl": "majority", "concept_idx": 2},
                        #   [{"cl": "majority", "concept_idx": 1}, {"cl": "majority", "concept_idx": 2}],
                        #   {"cl": "majority", "concept_idx": 4},
                        #   {"cl": "majority", "concept_idx": 5},
                        #   [{"cl": "majority", "concept_idx": 4}, {"cl": "majority", "concept_idx": 5}],
                          [{"cl": "majority", "concept_idx": 2}, {"cl": "majority", "concept_idx": 4}, {"cl": "majority", "concept_idx": 5}],
]

In [ ]:
# Iterate over possible shift type, conduct experimentation, store results
for shift_type, shift_type_params in zip(shift_type_list, shift_type_params_list):
    dict_result = dict()

    # Conduct experimentation
    dict_result["test"] = do_experiment_per_shift(multitask_model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    dict_result["valid"] = do_experiment_per_shift(multitask_model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    
    # Save result
    save_dict_result(shift_type, method, dict_result)


Saving successfully.


#### Image Shift

We consider the following concept shift combinations:
- rotation
- shear
- zoom
- translation (x)
- translation (y)
- translation (x, y)
- img (combination of all shifts)

In [ ]:
# List of shift_type and params
shift_type_list = [
                   "rotation",
                   "shear",
                   "zoom",
                   "width_shift",
                   "height_shift",
                   ["width_shift", "height_shift"],
                   "img"
]

shift_type_params_list = [
                          {"orig_dims": ORIGINAL_SHAPE},
                          {"orig_dims": ORIGINAL_SHAPE},
                          {"orig_dims": ORIGINAL_SHAPE},
                          {"orig_dims": ORIGINAL_SHAPE},
                          {"orig_dims": ORIGINAL_SHAPE},
                          [{"orig_dims": ORIGINAL_SHAPE}, {"orig_dims": ORIGINAL_SHAPE}],
                          {"orig_dims": ORIGINAL_SHAPE},
]

In [ ]:
# Iterate over possible shift type, conduct experimentation, store results
for shift_type, shift_type_params in zip(shift_type_list, shift_type_params_list):
    dict_result = dict()

    # Conduct experimentation
    dict_result["test"] = do_experiment_per_shift(multitask_model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    dict_result["valid"] = do_experiment_per_shift(multitask_model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    
    # Save result
    save_dict_result(shift_type, method, dict_result)


Saving successfully.



Saving successfully.



Saving successfully.



Saving successfully.



Saving successfully.



Saving successfully.



Saving successfully.


### BBSDs
Standard BBSDs, use the softmax outputs of the final prediction task as the reduced representation.

In [ ]:
method = "BBSDs"

In [ ]:
# Wrap the concept bottleneck model into and end-to-end model
# It is not necessary to use CBM, but we will use it for
# fair comparison.
model = FullModel(multitask_model, com)

#### Knockout Shift

In [ ]:
shift_type = "ko"
shift_type_params = {"cl": 0}

In [ ]:
dict_result = dict()
dict_result["test"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
dict_result["valid"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)

In [ ]:
save_dict_result(shift_type, method, dict_result)

Saving successfully.


#### Gaussian Shift

In [ ]:
shift_type = "gaussian"
shift_type_params = None

In [ ]:
dict_result = dict()
dict_result["test"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
dict_result["valid"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)

In [ ]:
save_dict_result(shift_type, method, dict_result)

Saving successfully.


#### Concept Shift

We consider the following concept shift combinations:
- Scale
- Scale + Shape
- Position X
- Position Y
- Position X + Position Y
- Position X + Position Y + Scale

In [ ]:
# List of shift_type and params
shift_type_list = ["concept_scale",
                   ["concept_shape", "concept_scale"],
                   "concept_x",
                   "concept_y",
                   ["concept_x", "concept_y"],
                   ["concept_x", "concept_y", "scale"],
]

shift_type_params_list = [{"cl": "majority", "concept_idx": 2},
                          [{"cl": "majority", "concept_idx": 1}, {"cl": "majority", "concept_idx": 2}],
                          {"cl": "majority", "concept_idx": 4},
                          {"cl": "majority", "concept_idx": 5},
                          [{"cl": "majority", "concept_idx": 4}, {"cl": "majority", "concept_idx": 5}],
                          [{"cl": "majority", "concept_idx": 2}, {"cl": "majority", "concept_idx": 4}, {"cl": "majority", "concept_idx": 5}],
]

In [ ]:
# Iterate over possible shift type, conduct experimentation, store results
for shift_type, shift_type_params in zip(shift_type_list, shift_type_params_list):
    dict_result = dict()

    # Conduct experimentation
    dict_result["test"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    dict_result["valid"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    
    # Save result
    save_dict_result(shift_type, method, dict_result)


Saving successfully.



Saving successfully.



Saving successfully.



Saving successfully.



Saving successfully.



Saving successfully.


#### Image Shift

We consider the following concept shift combinations:
- rotation
- shear
- zoom
- translation (x)
- translation (y)
- translation (x, y)
- img (combination of all shifts)

In [ ]:
# List of shift_type and params
shift_type_list = [
                   "rotation",
                   "shear",
                   "zoom",
                   "width_shift",
                   "height_shift",
                   ["width_shift", "height_shift"],
                   "img"
]

shift_type_params_list = [
                          {"orig_dims": ORIGINAL_SHAPE},
                          {"orig_dims": ORIGINAL_SHAPE},
                          {"orig_dims": ORIGINAL_SHAPE},
                          {"orig_dims": ORIGINAL_SHAPE},
                          {"orig_dims": ORIGINAL_SHAPE},
                          [{"orig_dims": ORIGINAL_SHAPE}, {"orig_dims": ORIGINAL_SHAPE}],
                          {"orig_dims": ORIGINAL_SHAPE},
]

In [ ]:
# Iterate over possible shift type, conduct experimentation, store results
for shift_type, shift_type_params in zip(shift_type_list, shift_type_params_list):
    dict_result = dict()

    # Conduct experimentation
    dict_result["test"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    dict_result["valid"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    
    # Save result
    save_dict_result(shift_type, method, dict_result)


Saving successfully.



Saving successfully.



Saving successfully.


### BBSDh
Standard BBSDh, use the hard prediction outputs of the final prediction task as the reduced representation.

In [ ]:
method = "BBSDh"

In [ ]:
# Wrap the concept bottleneck model into and end-to-end model
# It is not necessary to use CBM, but we will use it for
# fair comparison.
model = FullModel(multitask_model, com)

#### Knockout Shift

In [ ]:
shift_type = "ko"
shift_type_params = {"cl": 0}

In [ ]:
dict_result = dict()
dict_result["test"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
dict_result["valid"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)

In [ ]:
save_dict_result(shift_type, method, dict_result)

Saving successfully.


#### Gaussian Shift

In [ ]:
shift_type = "gaussian"
shift_type_params = None

In [ ]:
dict_result = dict()
dict_result["test"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
dict_result["valid"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)

In [ ]:
save_dict_result(shift_type, method, dict_result)

Saving successfully.


#### Concept Shift

We consider the following concept shift combinations:
- Scale
- Scale + Shape
- Position X
- Position Y
- Position X + Position Y
- Position X + Position Y + Scale

In [ ]:
# List of shift_type and params
shift_type_list = ["concept_scale",
                   ["concept_shape", "concept_scale"],
                   "concept_x",
                   "concept_y",
                   ["concept_x", "concept_y"],
                   ["concept_x", "concept_y", "scale"],
]

shift_type_params_list = [{"cl": "majority", "concept_idx": 2},
                          [{"cl": "majority", "concept_idx": 1}, {"cl": "majority", "concept_idx": 2}],
                          {"cl": "majority", "concept_idx": 4},
                          {"cl": "majority", "concept_idx": 5},
                          [{"cl": "majority", "concept_idx": 4}, {"cl": "majority", "concept_idx": 5}],
                          [{"cl": "majority", "concept_idx": 2}, {"cl": "majority", "concept_idx": 4}, {"cl": "majority", "concept_idx": 5}],
]

In [ ]:
# Iterate over possible shift type, conduct experimentation, store results
for shift_type, shift_type_params in zip(shift_type_list, shift_type_params_list):
    dict_result = dict()

    # Conduct experimentation
    dict_result["test"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    dict_result["valid"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    
    # Save result
    save_dict_result(shift_type, method, dict_result)


Saving successfully.



Saving successfully.



Saving successfully.



Saving successfully.



Saving successfully.



Saving successfully.


#### Image Shift

We consider the following concept shift combinations:
- rotation
- shear
- zoom
- translation (x)
- translation (y)
- translation (x, y)
- img (combination of all shifts)

In [ ]:
# List of shift_type and params
shift_type_list = [
                   "rotation",
                   "shear",
                   "zoom",
                   "width_shift",
                   "height_shift",
                   ["width_shift", "height_shift"],
                   "img"
]

shift_type_params_list = [
                          {"orig_dims": ORIGINAL_SHAPE},
                          {"orig_dims": ORIGINAL_SHAPE},
                          {"orig_dims": ORIGINAL_SHAPE},
                          {"orig_dims": ORIGINAL_SHAPE},
                          {"orig_dims": ORIGINAL_SHAPE},
                          [{"orig_dims": ORIGINAL_SHAPE}, {"orig_dims": ORIGINAL_SHAPE}],
                          {"orig_dims": ORIGINAL_SHAPE},
]

In [ ]:
# Iterate over possible shift type, conduct experimentation, store results
for shift_type, shift_type_params in zip(shift_type_list, shift_type_params_list):
    dict_result = dict()

    # Conduct experimentation
    dict_result["test"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    dict_result["valid"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    
    # Save result
    save_dict_result(shift_type, method, dict_result)

### PCA

Use the original features principal components as reduced representation for shift detection.

In [25]:
method = "PCA"

In [24]:
# The model for dimensionality reduction
model = pca

#### Knockout Shift

In [17]:
shift_type = "ko"
shift_type_params = {"cl": "majority"}

In [18]:
dict_result = dict()
dict_result["test"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
dict_result["valid"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)

In [19]:
save_dict_result(shift_type, method, dict_result)

Saving successfully.


#### Gaussian Shift

In [ ]:
shift_type = "gaussian"
shift_type_params = None

In [ ]:
dict_result = dict()
dict_result["test"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
dict_result["valid"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)

In [ ]:
save_dict_result(shift_type, method, dict_result)

Saving successfully.


#### Concept Shift

We consider the following concept shift combinations:
- Scale
- Scale + Shape
- Position X
- Position Y
- Position X + Position Y
- Position X + Position Y + Scale

In [26]:
# List of shift_type and params
shift_type_list = [
                   "concept_scale",
                #    ["concept_shape", "concept_scale"],
                #    "concept_x",
                #    "concept_y",
                #    ["concept_x", "concept_y"],
                #    ["concept_x", "concept_y", "scale"],
]

shift_type_params_list = [
                          {"cl": "majority", "concept_idx": 2},
                        #   [{"cl": "majority", "concept_idx": 1}, {"cl": "majority", "concept_idx": 2}],
                        #   {"cl": "majority", "concept_idx": 4},
                        #   {"cl": "majority", "concept_idx": 5},
                        #   [{"cl": "majority", "concept_idx": 4}, {"cl": "majority", "concept_idx": 5}],
                        #   [{"cl": "majority", "concept_idx": 2}, {"cl": "majority", "concept_idx": 4}, {"cl": "majority", "concept_idx": 5}],
]

In [27]:
# Iterate over possible shift type, conduct experimentation, store results
for shift_type, shift_type_params in zip(shift_type_list, shift_type_params_list):
    dict_result = dict()

    # Conduct experimentation
    dict_result["test"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    dict_result["valid"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    
    # Save result
    save_dict_result(shift_type, method, dict_result)


Saving successfully.


#### Image Shift

We consider the following concept shift combinations:
- rotation
- shear
- zoom
- translation (x)
- translation (y)
- translation (x, y)
- img (combination of all shifts)

In [ ]:
# List of shift_type and params
shift_type_list = [
                #    "rotation",
                #    "shear",
                #    "zoom",
                #    "width_shift",
                #    "height_shift",
                   ["width_shift", "height_shift"],
                   "img"
]

shift_type_params_list = [
                        #   {"orig_dims": ORIGINAL_SHAPE},
                        #   {"orig_dims": ORIGINAL_SHAPE},
                        #   {"orig_dims": ORIGINAL_SHAPE},
                        #   {"orig_dims": ORIGINAL_SHAPE},
                        #   {"orig_dims": ORIGINAL_SHAPE},
                          [{"orig_dims": ORIGINAL_SHAPE}, {"orig_dims": ORIGINAL_SHAPE}],
                          {"orig_dims": ORIGINAL_SHAPE},
]

In [ ]:
# Iterate over possible shift type, conduct experimentation, store results
for shift_type, shift_type_params in zip(shift_type_list, shift_type_params_list):
    dict_result = dict()

    # Conduct experimentation
    dict_result["test"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    dict_result["valid"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    
    # Save result
    save_dict_result(shift_type, method, dict_result)


Saving successfully.



Saving successfully.


### SRP

We use the sparse random projection as the reduced representation of the feature.

In [ ]:
method = "SRP"

In [ ]:
# The model for dimensionality reduction
model = srp

#### Knockout Shift

In [ ]:
shift_type = "ko"
shift_type_params = {"cl": "majority"}

In [ ]:
dict_result = dict()
dict_result["test"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
dict_result["valid"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)

In [ ]:
save_dict_result(shift_type, method, dict_result)

Saving successfully.


#### Gaussian Shift

In [ ]:
shift_type = "gaussian"
shift_type_params = None

In [ ]:
dict_result = dict()
dict_result["test"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
dict_result["valid"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)

In [ ]:
save_dict_result(shift_type, method, dict_result)

Saving successfully.


#### Concept Shift

We consider the following concept shift combinations:
- Scale
- Scale + Shape
- Position X
- Position Y
- Position X + Position Y
- Position X + Position Y + Scale

In [ ]:
# List of shift_type and params
shift_type_list = [
                   "concept_scale",
                #    ["concept_shape", "concept_scale"],
                #    "concept_x",
                #    "concept_y",
                #    ["concept_x", "concept_y"],
                #    ["concept_x", "concept_y", "scale"],
]

shift_type_params_list = [
                          {"cl": "majority", "concept_idx": 2},
                        #   [{"cl": "majority", "concept_idx": 1}, {"cl": "majority", "concept_idx": 2}],
                        #   {"cl": "majority", "concept_idx": 4},
                        #   {"cl": "majority", "concept_idx": 5},
                        #   [{"cl": "majority", "concept_idx": 4}, {"cl": "majority", "concept_idx": 5}],
                        #   [{"cl": "majority", "concept_idx": 2}, {"cl": "majority", "concept_idx": 4}, {"cl": "majority", "concept_idx": 5}],
]

In [ ]:
# Iterate over possible shift type, conduct experimentation, store results
for shift_type, shift_type_params in zip(shift_type_list, shift_type_params_list):
    dict_result = dict()

    # Conduct experimentation
    dict_result["test"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    dict_result["valid"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    
    # Save result
    save_dict_result(shift_type, method, dict_result)


Saving successfully.


#### Image Shift

We consider the following concept shift combinations:
- rotation
- shear
- zoom
- translation (x)
- translation (y)
- translation (x, y)
- img (combination of all shifts)

In [ ]:
# List of shift_type and params
shift_type_list = [
                #    "rotation",
                #    "shear",
                #    "zoom",
                #    "width_shift",
                #    "height_shift",
                   ["width_shift", "height_shift"],
                   "img"
]

shift_type_params_list = [
                        #   {"orig_dims": ORIGINAL_SHAPE},
                        #   {"orig_dims": ORIGINAL_SHAPE},
                        #   {"orig_dims": ORIGINAL_SHAPE},
                        #   {"orig_dims": ORIGINAL_SHAPE},
                        #   {"orig_dims": ORIGINAL_SHAPE},
                          [{"orig_dims": ORIGINAL_SHAPE}, {"orig_dims": ORIGINAL_SHAPE}],
                          {"orig_dims": ORIGINAL_SHAPE},
]

In [ ]:
# Iterate over possible shift type, conduct experimentation, store results
for shift_type, shift_type_params in zip(shift_type_list, shift_type_params_list):
    dict_result = dict()

    # Conduct experimentation
    dict_result["test"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_test_flatten, y_test, c_test, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    dict_result["valid"] = do_experiment_per_shift(model, method, x_valid, y_valid,
                        c_valid, x_valid_flatten, y_valid, c_valid, shift_type, ORIGINAL_SHAPE,
                        shift_type_params)
    
    # Save result
    save_dict_result(shift_type, method, dict_result)


Saving successfully.



Saving successfully.
